![](https://github.com/destination-earth/DestinE-DataLake-Lab/blob/main/img/DestinE-banner.jpg?raw=true)

**Author**: EUMETSAT <br>
**Copyright**: 2024 EUMETSAT <br>
**Licence**: MIT <br>

# HDA PySTAC-Client Introduction


This notebook shows the basic use of DestinE Data Lake Harmonised Data Access using pystac-client.
It will include iterating through Collections and Items, and perform simple spatio-temporal searches.

## Obtain DEDL Access Token to use the HDA service

In [1]:
import requests
import json
import os
from getpass import getpass
import destinelab as deauth

In [2]:
DESP_USERNAME = input("Please input your DESP username or email: ")
DESP_PASSWORD = getpass("Please input your DESP password: ")

auth = deauth.AuthHandler(DESP_USERNAME, DESP_PASSWORD)
access_token = auth.get_token()
if access_token is not None:
    print("DEDL/DESP Access Token Obtained Successfully")
else:
    print("Failed to Obtain DEDL/DESP Access Token")

auth_headers = {"Authorization": f"Bearer {access_token}"}

Please input your DESP username or email:  eum-dedl-user
Please input your DESP password:  ········


Response code: 200
DEDL/DESP Access Token Obtained Successfully


## Set username and password as environment variables to be used for DEDL data access

In [3]:
import os

os.environ["EODAG__DEDL__AUTH__CREDENTIALS__USERNAME"] = DESP_USERNAME
os.environ["EODAG__DEDL__AUTH__CREDENTIALS__PASSWORD"] = DESP_PASSWORD

# Create pystac client object for HDA STAC API
We first connect to an API by retrieving the root catalog, or landing page, of the API with the Client.open function.

In [4]:
from pystac_client import Client

HDA_API_URL = "https://hda.data.destination-earth.eu/stac"
cat = Client.open(HDA_API_URL, headers=auth_headers)

## Query all available collections
As with a static catalog the get_collections function will iterate through the Collections in the Catalog. 
Notice that because this is an API it can get all the Collections through a single call, rather than having to fetch each one individually.

In [5]:
from rich.console import Console
import rich.table

console = Console()

hda_collections = cat.get_collections()

table = rich.table.Table(title="HDA collections", expand=True)
table.add_column("ID", style="cyan", justify="right",no_wrap=True)
table.add_column("Title", style="violet", no_wrap=True)
for collection in hda_collections:
    table.add_row(collection.id, collection.title)
console.print(table)

                                                  HDA collections                                                  
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃                                      ID ┃ Title                                                                 ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ EO.ECMWF.DAT.REANALYSIS_ERA5_SINGLE_LE… │ ERA5 hourly data on single levels from 1940 to present                │
│          EO.DEM.DAT.COP-DEM_GLO-90-DGED │ COPERNICUS Digital Elevation Model (DEM) 90 meters DGED               │
│ EO.ECMWF.DAT.SEASONAL_FORECAST_ANOMALI… │ Seasonal forecast anomalies on single levels                          │
│               EO.EUM.DAT.SENTINEL-3.AOD │ SLSTR Level 2 Aerosol Optical Depth - Sentinel-3                      │
│       EO.ECMWF.DAT.CEMS_FIRE_HISTORICAL │ Fire danger indices historical data from the Copernicus Emergency Ma… │
│ STAT.EUSTAT.DAT.POP_CHANGE_DEMO_BALANC… │ Eurostat - Population change - Demographic balance and crude rates a… │
│ EO.ECMWF.DAT.SEA_LEVEL_DAILY_GRIDDED_D… │ Sea level gridded data from satellite observations for the global oc… │
│ EO.ECMWF.DAT.CAMS_GLOBAL_ATMOSHERIC_CO… │ CAMS global atmospheric composition forecasts                         │
│       EO.ECMWF.DAT.CEMS_GLOFAS_SEASONAL │ Seasonal forecasts of river discharge and related data by the Global… │
│ EO.ECMWF.DAT.CEMS_GLOFAS_SEASONAL_REFO… │ Seasonal reforecasts of river discharge and related data from the Gl… │
│ EO.ECMWF.DAT.ERA5_MONTHLY_MEANS_VARIAB… │ ERA5 monthly averaged data on pressure levels from 1940 to present    │
│ EO.ECMWF.DAT.CAMS_GLOBAL_REANALYSIS_EA… │ CAMS global reanalysis (EAC4)                                         │
│ EO.ECMWF.DAT.CO2_DATA_FROM_SATELLITE_S… │ Carbon dioxide data from 2002 to present derived from satellite obse… │
│       EO.EUM.DAT.SENTINEL-3.SR_1_SRA_A_ │ SRAL Level 1B - Sentinel-3                                            │
│ STAT.EUSTAT.DAT.SHARE_ENERGY_FROM_RENE… │ Eurostat - Share of energy from renewable sources                     │
│       EO.ESA.DAT.SENTINEL-3.OL_2_LRR___ │ OLCI Land Colour Reduced Resolution - Sentinel-3                      │
│       EO.ESA.DAT.SENTINEL-3.OL_2_LFR___ │ OLCI Land Colour Full Resolution - Sentinel-3                         │
│     EO.ECMWF.DAT.CEMS_GLOFAS_REFORECAST │ Reforecasts of river discharge and related data by the Global Flood … │
│       STAT.EUSTAT.DAT.POP_AGE_SEX_NUTS2 │ Population on 1 January by age, sex and NUTS 2 region                 │
│ EO.ECMWF.DAT.SEASONAL_FORECAST_MONTHLY… │ Seasonal forecast monthly statistics on single levels                 │
│ EO.ECMWF.DAT.GLACIERS_DISTRIBUTION_DAT… │ Glaciers distribution data from the Randolph Glacier Inventory for y… │
│           EO.ESA.DAT.SENTINEL-2.MSI.L2A │ Sentinel 2 MSI Level 2-A                                              │
│              EO.ECMWF.DAT.EFAS_FORECAST │ River discharge and related forecasted data by the European Flood Aw… │
│              EO.ECMWF.DAT.EFAS_SEASONAL │ Seasonal forecasts of river discharge and related data by the Europe… │
│ EO.ECMWF.DAT.SIS_HYDROLOGY_METEOROLOGY… │ Temperature and precipitation climate impact indicators from 1970 to… │
│ STAT.EUSTAT.DAT.POP_AGE_GROUP_SEX_NUTS3 │ Population on 1 January by age, sex and NUTS 3 region                 │
│       EO.EUM.DAT.SENTINEL-3.OL_1_ERR___ │ OLCI Level 1B Reduced Resolution - Sentinel-3                         │
│ EO.ECMWF.DAT.SEA_LEVEL_DAILY_GRIDDED_D… │ Sea level daily gridded data from satellite observations for the Bla… │
│ EO.ECMWF.DAT.CAMS_GREENHOUSE_GAS_FLUXES │ CAMS global inversion-optimised greenhouse gas fluxes and concentrat… │
│       EO.ESA.DAT.SENTINEL-5P.TROPOMI.L1 │ SENTINEL-5P TROPOMI Level 1B                                          │
│   EO.ECMWF.DAT.EFAS_SEASONAL_REFORECAST │ Seasonal ref

## Obtain provider information for each individual collection

In [6]:
table = rich.table.Table(title="HDA collections | Providers", expand=True)
table.add_column("Title", style="cyan", justify="right", no_wrap=True)
table.add_column("Provider", style="violet", no_wrap=True)

hda_collections = cat.get_collections()

for collection in hda_collections:
    collection_details = cat.get_collection(collection.id)
    provider = ','.join(str(x.name) for x in collection_details.providers)
    table.add_row(collection_details.title, provider)
console.print(table)

                                            HDA collections | Providers                                            
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃                                                  Title ┃ Provider                                               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ ERA5 hourly data on single levels from 1940 to present │ European Centre for Medium-Range Weather Forecasts (E… │
│ COPERNICUS Digital Elevation Model (DEM) 90 meters DG… │ ESA,creodias,wekeo                                     │
│           Seasonal forecast anomalies on single levels │ European Centre for Medium-Range Weather Forecasts (E… │
│       SLSTR Level 2 Aerosol Optical Depth - Sentinel-3 │ EUMETSAT,creodias,dedl                                 │
│ Fire danger indices historical data from the Copernic… │ European Centre for Medium-Range Weather Forecasts (E… │
│ Eurostat - Population change - Demographic balance an… │ Eurostat,dedl                                          │
│ Sea level gridded data from satellite observations fo… │ European Centre for Medium-Range Weather Forecasts (E… │
│          CAMS global atmospheric composition forecasts │ European Centre for Medium-Range Weather Forecasts (E… │
│ Seasonal forecasts of river discharge and related dat… │ European Centre for Medium-Range Weather Forecasts (E… │
│ Seasonal reforecasts of river discharge and related d… │ European Centre for Medium-Range Weather Forecasts (E… │
│ ERA5 monthly averaged data on pressure levels from 19… │ European Centre for Medium-Range Weather Forecasts (E… │
│                          CAMS global reanalysis (EAC4) │ European Centre for Medium-Range Weather Forecasts (E… │
│ Carbon dioxide data from 2002 to present derived from… │ European Centre for Medium-Range Weather Forecasts (E… │
│                             SRAL Level 1B - Sentinel-3 │ EUMETSAT,creodias,wekeo,dedl                           │
│      Eurostat - Share of energy from renewable sources │ Eurostat,dedl                                          │
│       OLCI Land Colour Reduced Resolution - Sentinel-3 │ ESA,creodias,wekeo,dedl                                │
│          OLCI Land Colour Full Resolution - Sentinel-3 │ ESA,creodias,wekeo,dedl                                │
│ Reforecasts of river discharge and related data by th… │ European Centre for Medium-Range Weather Forecasts (E… │
│  Population on 1 January by age, sex and NUTS 2 region │ Eurostat,dedl                                          │
│  Seasonal forecast monthly statistics on single levels │ European Centre for Medium-Range Weather Forecasts (E… │
│ Glaciers distribution data from the Randolph Glacier … │ European Centre for Medium-Range Weather Forecasts (E… │
│                               Sentinel 2 MSI Level 2-A │ ESA,creodias,wekeo,dedl                                │
│ River discharge and related forecasted data by the Eu… │ European Centre for Medium-Range Weather Forecasts (E… │
│ Seasonal forecasts of river discharge and related dat… │ European Centre for Medium-Range Weather Forecasts (E… │
│ Temperature and precipitation climate impact indicato… │ European Centre for Medium-Range Weather Forecasts (E… │
│  Population on 1 January by age, sex and NUTS 3 region │ Eurostat,dedl                                          │
│          OLCI Level 1B Reduced Resolution - Sentinel-3 │ EUMETSAT,creodias,wekeo,dedl                           │
│ Sea level daily gridded data from satellite observati… │ European Centre for Medium-Range Weather Forecasts (E… │
│ CAMS global inversion-optimised greenhouse gas fluxes… │ European Centre for Medium-Range Weather Forecasts (E… │
│                           SENTINEL-5P TROPOMI Level 1B │ ESA,creodias,dedl                                      │
│ Seasonal reforecasts of river discharge and related d…

## Inspect Items of a Collection
The main functions for getting items return iterators, where pystac-client will handle retrieval of additional pages when needed. Note that one request is made for the first ten items, then a second request for the next ten.

In [7]:
coll_name = 'EO.ESA.DAT.SENTINEL-1.L1_GRD'
search = cat.search(
    max_items=10,
    collections=[coll_name],
    bbox=[-72.5,40.5,-72,41],
    datetime="2023-09-09T00:00:00Z/2023-09-20T23:59:59Z"
)

coll_items = search.item_collection()
console.print(f"For collection {coll_name} we found {len(coll_items)} items")

For collection EO.ESA.DAT.SENTINEL-1.L1_GRD we found 4 items

In [8]:
import geopandas

df = geopandas.GeoDataFrame.from_features(coll_items.to_dict(), crs="epsg:4326")
df.head()



,geometry,providers,datetime,start_datetime,end_datetime,updated,description,constellation,platform,instruments,...,eo:snow_cover,processing:level,sat:orbit_state,sat:relative_orbit,sat:absolute_orbit,sar:product_type,sar:instrument_mode,oseo:accessConstraint,oseo:swathIdentifier,oseo:polarizationMode
0,"POLYGON ((-72.73724 38.96255, -69.78494 39.365...","[{'name': 'dedl', 'description': 'DEDL Fresh D...",2023-09-09T22:43:00.443000Z,2023-09-09T22:43:00.443Z,2023-09-09T22:43:25.442Z,2023-09-10T03:08:53.952Z,The Sentinel-1 mission is the European Radar O...,SENTINEL-1,S1A,[SAR],...,0,LEVEL1,ASCENDING,135,50257,IW_GRDH_1S,IW,"{'licenseId': 'unlicensed', 'hasToBeSigned': '...",IW,VV&VH
1,"POLYGON ((-73.11689 40.46453, -70.09798 40.866...","[{'name': 'dedl', 'description': 'DEDL Fresh D...",2023-09-09T22:43:25.443000Z,2023-09-09T22:43:25.443Z,2023-09-09T22:43:50.442Z,2023-09-10T02:57:33.550Z,The Sentinel-1 mission is the European Radar O...,SENTINEL-1,S1A,[SAR],...,0,LEVEL1,ASCENDING,135,50257,IW_GRDH_1S,IW,"{'licenseId': 'unlicensed', 'hasToBeSigned': '...",IW,VV&VH
2,"POLYGON ((-74.73586 38.73753, -71.84171 39.134...","[{'name': 'dedl', 'description': 'DEDL Fresh D...",2023-09-14T22:51:10.454000Z,2023-09-14T22:51:10.454Z,2023-09-14T22:51:35.452Z,2023-09-15T01:30:12.125Z,The Sentinel-1 mission is the European Radar O...,SENTINEL-1,S1A,[SAR],...,0,LEVEL1,ASCENDING,33,50330,IW_GRDH_1S,IW,"{'licenseId': 'unlicensed', 'hasToBeSigned': '...",IW,VV&VH
3,"POLYGON ((-75.11328 40.23970, -72.15421 40.635...","[{'name': 'dedl', 'description': 'DEDL Fresh D...",2023-09-14T22:51:35.454000Z,2023-09-14T22:51:35.454Z,2023-09-14T22:52:00.453Z,2023-09-15T01:28:26.349Z,The Sentinel-1 mission is the European Radar O...,SENTINEL-1,S1A,[SAR],...,0,LEVEL1,ASCENDING,33,50330,IW_GRDH_1S,IW,"{'licenseId': 'unlicensed', 'hasToBeSigned': '...",IW,VV&VH


## Inspect STAC assets of an item

In [9]:
import rich.table

selected_item = coll_items[3]

table = rich.table.Table(title="Assets in STAC Item")
table.add_column("Asset Key", style="cyan", no_wrap=True)
table.add_column("Description")
for asset_key, asset in selected_item.assets.items():
    table.add_row(asset_key, asset.title)

console.print(table)

                                                Assets in STAC Item                                                
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━┓
┃ Asset Key                                                                                           ┃ Descript… ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━┩
│ downloadLink                                                                                        │ Download  │
│                                                                                                     │ link      │
│ S1A_IW_GRDH_1SDV_20230914T225135_20230914T225200_050330_060F40_9F54.SAFE-report-20230915T011838.pdf │ S1A_IW_G… │
│ calibration-s1a-iw-grd-vh-20230914t225135-20230914t225200-050330-060f40-002.xml                     │ calibrat… │
│ calibration-s1a-iw-grd-vv-20230914t225135-20230914t225200-050330-060f40-001.xml                     │ calibrat… │
│ noise-s1a-iw-grd-vh-20230914t225135-20230914t225200-050330-060f40-002.xml                           │ noise-s1… │
│ noise-s1a-iw-grd-vv-20230914t225135-20230914t225200-050330-060f40-001.xml                           │ noise-s1… │
│ rfi-s1a-iw-grd-vh-20230914t225135-20230914t225200-050330-060f40-002.xml                             │ rfi-s1a-… │
│ rfi-s1a-iw-grd-vv-20230914t225135-20230914t225200-050330-060f40-001.xml                             │ rfi-s1a-… │
│ s1a-iw-grd-vh-20230914t225135-20230914t225200-050330-060f40-002.xml                                 │ s1a-iw-g… │
│ s1a-iw-grd-vv-20230914t225135-20230914t225200-050330-060f40-001.xml                                 │ s1a-iw-g… │
│ manifest.safe                                                                                       │ manifest… │
│ s1a-iw-grd-vh-20230914t225135-20230914t225200-050330-060f40-002.tiff                                │ s1a-iw-g… │
│ s1a-iw-grd-vv-20230914t225135-20230914t225200-050330-060f40-001.tiff                                │ s1a-iw-g… │
│ logo.png                                                                                            │ logo.png  │
│ map-overlay.kml                                                                                     │ map-over… │
│ product-preview.html                                                                                │ product-… │
│ quick-look.png                                                                                      │ quick-lo… │
│ thumbnail.png                                                                                       │ thumbnai… │
│ s1-level-1-calibration.xsd                                                                          │ s1-level… │
│ s1-level-1-measurement.xsd                                                                          │ s1-level… │
│ s1-level-1-noise.xsd                                                                                │ s1-level… │
│ s1-level-1-product.xsd                                                                              │ s1-level… │
│ s1-level-1-quicklook.xsd                                                                            │ s1-level… │
│ s1-level-1-rfi.xsd                                                                                  │ s1-level… │
│ s1-map-overlay.xsd                                                                                  │ s1-map-o… │
│ s1-object-types.xsd                                                                                 │ s1-objec… │
│ s1-product-preview.xsd                                                                              │ s1-produ… │
│ thumbnail                                                                                           │ Thumbnail │
└─────────────────────────────────────────────────────────────────────────────────────────────────────┴───────────┘

In [10]:
from IPython.display import Image

Image(url=selected_item.assets["thumbnail"].href, width=500)

In [11]:
down_uri = selected_item.assets["downloadLink"].href
console.print(f"Download link of asset is {down_uri}")

Download link of asset is 
https://hda.data.destination-earth.eu/stac/collections/EO.ESA.DAT.SENTINEL-1.L1_GRD/items/S1A_IW_GRDH_1SDV_20230914
T225135_20230914T225200_050330_060F40_9F54/download?provider=dedl

### Download asset to JupyterLab

In [12]:
selected_item.id

'S1A_IW_GRDH_1SDV_20230914T225135_20230914T225200_050330_060F40_9F54'

In [13]:
selected_item.assets["downloadLink"]

<Asset href=https://hda.data.destination-earth.eu/stac/collections/EO.ESA.DAT.SENTINEL-1.L1_GRD/items/S1A_IW_GRDH_1SDV_20230914T225135_20230914T225200_050330_060F40_9F54/download?provider=dedl>

In [ ]:
# Make http request for remote file data
data = requests.get(selected_item.assets["downloadLink"].href,
                   headers=auth_headers)
mtype = selected_item.assets["downloadLink"].media_type.split("/")[1]
# Save file data to local copy
with open(f"{selected_item.id}.{mtype}", 'wb')as file:
    file.write(data.content)